# Importing libraries and datasets

In [0]:
# importing basic datasets
import numpy  as np
import pandas as pd

In [35]:
# importing the census 2011 data
cen = pd.read_csv('population_india_census2011.csv')
cen.head()

,Sno,State / Union Territory,Population,Rural population,Urban population,Area,Density,Gender Ratio
0,1,Uttar Pradesh,199812341,155317278,44495063,"240,928 km2 (93,023 sq mi)","828/km2 (2,140/sq mi)",912
1,2,Maharashtra,112374333,61556074,50818259,"307,713 km2 (118,809 sq mi)",365/km2 (950/sq mi),929
2,3,Bihar,104099452,92341436,11758016,"94,163 km2 (36,357 sq mi)","1,102/km2 (2,850/sq mi)",918
3,4,West Bengal,91276115,62183113,29093002,"88,752 km2 (34,267 sq mi)","1,029/km2 (2,670/sq mi)",953
4,5,Madhya Pradesh,72626809,52557404,20069405,"308,245 km2 (119,014 sq mi)",236/km2 (610/sq mi),931


In [36]:
# importing the covid india dataset
cov = pd.read_csv('covid_19_india.csv')
cov.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,6:00 PM,Kerala,1,0,0.0,0.0,1
1,2,31/01/20,6:00 PM,Kerala,1,0,0.0,0.0,1
2,3,01/02/20,6:00 PM,Kerala,2,0,0.0,0.0,2
3,4,02/02/20,6:00 PM,Kerala,3,0,0.0,0.0,3
4,5,03/02/20,6:00 PM,Kerala,3,0,0.0,0.0,3


# Data Cleaning

### checking columns

In [37]:
# checing census column names.
cen.columns

Index(['Sno', 'State / Union Territory', 'Population', 'Rural population',
       'Urban population', 'Area', 'Density', 'Gender Ratio'],
      dtype='object')

In [3]:
# checking covid 19 column names
cov.columns

Index(['Sno', 'Date', 'Time', 'State/UnionTerritory',
       'ConfirmedIndianNational', 'ConfirmedForeignNational', 'Cured',
       'Deaths', 'Confirmed'],
      dtype='object')

In [4]:
# checking the unique values in Time column to ascertain the value of th column.
cov.Time.unique()

array(['6:00 PM', '10:00 AM', '7:30 PM', '9:30 PM', '8:30 PM', '5:00 PM',
       '8:00 AM'], dtype=object)

**Observations**

- covid 19 dataframe

  - The 'SNo' column is redundant.

  - The 'Time' column notes the time of the case being confirmed. This can be dropped as it won't be used in this particular analysis although it does serve the purpose of showing at what times were the most cases reported.

  - The point of having separate columns for Indian and Foreign nationals is redundant as the number of cases is far beyond for it to matter. A combined column showing the number of confirmed will suffce.

- census dataframe

  - The 'SNo' column is redundant.

  - The 'State / Union Territory' column name will have to be changed to match to the other dataframe.

  - The 'Rural population' and 'Urban population' column can be dropped as they are not going to be used in this analysis and the 'population' column in itself is sufficient.


### dropping unnessacary columns

In [0]:
# dropping the columns not necessary for the analysis.

cov.drop([
          'Sno',
          'Time',
          'ConfirmedIndianNational',
          'ConfirmedForeignNational'
          ],
         axis=1, 
         inplace=True)

cen.drop([
          'Sno',
          'Rural population',
          'Urban population'
          ],axis=1,
         inplace=True)

# renaming the column name for 'State / Union Territory' in census dataframe.

cen.columns = [
               'State/UnionTerritory', 
               'Population', 
               'Area', 
               'Density', 
               'GenderRatio'
               ]

### treating null values and data types

In [6]:
# checking null values
cov.isna().sum()

Date                    0
State/UnionTerritory    0
Cured                   1
Deaths                  1
Confirmed               0
dtype: int64

In [38]:
cen.isna().sum()

Sno                        0
State / Union Territory    0
Population                 0
Rural population           0
Urban population           0
Area                       0
Density                    0
Gender Ratio               0
dtype: int64

In [7]:
cov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2134 entries, 0 to 2133
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  2134 non-null   object 
 1   State/UnionTerritory  2134 non-null   object 
 2   Cured                 2133 non-null   float64
 3   Deaths                2133 non-null   float64
 4   Confirmed             2134 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 83.5+ KB


**Observations**

- The NaN values can be replaced by 0 as these are confirmed non-assigned cases which will be added to the states numbers.

- Cured and Deaths column can be converted to int from float.

In [0]:
cov.fillna(0,inplace=True)

In [0]:
cov.Cured = cov.Cured.astype(int)
cov.Deaths = cov.Deaths.astype(int)

### correcting errors

In [10]:
cov['State/UnionTerritory'].unique(), len(cov['State/UnionTerritory'].unique())

# There are a few errors in the names.
# 'Nagaland#', 'Nagaland'
# 'Jharkhand#', 'Jharkhand'
# 'Dadar Nagar Haveli', 'Dadra and Nagar Haveli and Daman and Diu'

(array(['Kerala', 'Telengana', 'Delhi', 'Rajasthan', 'Uttar Pradesh',
        'Haryana', 'Ladakh', 'Tamil Nadu', 'Karnataka', 'Maharashtra',
        'Punjab', 'Jammu and Kashmir', 'Andhra Pradesh', 'Uttarakhand',
        'Odisha', 'Puducherry', 'West Bengal', 'Chhattisgarh',
        'Chandigarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh',
        'Bihar', 'Manipur', 'Mizoram', 'Andaman and Nicobar Islands',
        'Goa', 'Unassigned', 'Assam', 'Jharkhand', 'Arunachal Pradesh',
        'Tripura', 'Nagaland', 'Meghalaya', 'Nagaland#', 'Jharkhand#',
        'Dadar Nagar Haveli', 'Cases being reassigned to states'],
       dtype=object), 38)

In [0]:
cov.replace('Nagaland#', 'Nagaland', inplace=True)
cov.replace('Jharkhand#', 'Jharkhand', inplace=True)
cov.replace('Dadar Nagar Haveli', 'Dadra and Nagar Haveli and Daman and Diu', inplace=True)

# EDA

## Confirmed cases ranking

In [12]:
# finding the states with the most confirmed cases
# grouping the data masking for most recent date
cov[
    cov['Date']==cov['Date'].unique()[-1]                           # masking by current date
    ].sort_values(
        'Confirmed',                                                # sorting descending by 'Confirmed' 
        ascending=False
        )[[
           'State/UnionTerritory',                                  # Showing only state name and confirmed columns
           'Confirmed'
           ]].style.background_gradient(
               cmap='Blues'                                         # choosing a color gradient for the table
           ).set_table_styles([
                               {'selector':'th',                    # choosing font size and weight for table header
                                'props':[('font-size','12px'),
                                         ('font-weight','bold')]},
                               {'selector':'td',                    # choosing font size and weight for table data
                                'props':[('font-size','11px'),
                                         ('font-weight','normal')]}
           ])

,State/UnionTerritory,Confirmed
2119,Maharashtra,37136
2127,Tamil Nadu,12448
2110,Gujarat,12140
2108,Delhi,10554
2126,Rajasthan,5845
2118,Madhya Pradesh,5465
2131,Uttar Pradesh,4926
2132,West Bengal,2961
2101,Andhra Pradesh,2532
2125,Punjab,2002


## Most Cured ranking

In [13]:
# ranking the states by the most cured cases
cov[
    cov['Date']==cov['Date'].unique()[-1]                           # masking by current date
    ].sort_values(
        'Cured',
        ascending=False                                             # sorting values by desceding order
        )[[
           'State/UnionTerritory',                                  # Showing only state name and cured columns
           'Cured'
           ]].style.background_gradient(
               cmap='Greens'                                        # choosing a color gradient for the table
           ).set_table_styles([
                               {'selector':'th',                    # choosing font size and weight for table header
                                'props':[('font-size','12px'),
                                         ('font-weight','bold')]},
                               {'selector':'td',                    # choosing font size and weight for table data
                                'props':[('font-size','11px'),      
                                         ('font-weight','normal')]}
           ])

,State/UnionTerritory,Cured
2119,Maharashtra,9639
2110,Gujarat,5043
2127,Tamil Nadu,4895
2108,Delhi,4750
2126,Rajasthan,3337
2131,Uttar Pradesh,2918
2118,Madhya Pradesh,2630
2125,Punjab,1642
2101,Andhra Pradesh,1621
2132,West Bengal,1074


In [14]:
cov[
    cov['Date']==cov['Date'].unique()[-1]
    ].sort_values(
        'Deaths',
        ascending=False
        )[[
           'State/UnionTerritory',
           'Deaths'
           ]].style.background_gradient(
               cmap='Reds'
           ).set_table_styles([
                               {'selector':'th',
                                'props':[('font-size','12px'),
                                         ('font-weight','bold')]},
                               {'selector':'td',
                                'props':[('font-size','11px'),
                                         ('font-weight','normal')]}
           ])

,State/UnionTerritory,Deaths
2119,Maharashtra,1325
2110,Gujarat,719
2118,Madhya Pradesh,258
2132,West Bengal,250
2108,Delhi,168
2126,Rajasthan,143
2131,Uttar Pradesh,123
2127,Tamil Nadu,84
2101,Andhra Pradesh,52
2115,Karnataka,40


**Observations**

- Maharashtra and Gujarat rank high on all the lists from the number of confirmed cases to recovered to number of fatalities.

- The majority of the confirmed cases are in Mahrastra, Tamil Nadu, Gujarat and Delhi.

- New metrics needs to be considered to get a more hollistic view of the data.

  - $Fatality$ $Rate = \frac{Deaths}{Confirmed}$

  - $Recovered$ $Rate = \frac{Recovered}{Confirmed}$

  - $Resilience = \frac{Recovered}{Deaths}$

  - $Spread = \frac{Confirmed}{Population}$

  - Effect of population density on the spread of the disease. 

In [0]:
crnt = cov[
    cov['Date']==cov['Date'].unique()[-1]
    ]

In [16]:
crnt['FatalityRate'] = crnt['Deaths']/crnt['Confirmed']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
crnt['RecoveryRate'] = crnt['Cured']/crnt['Confirmed']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
crnt.sort_values(
    'FatalityRate',ascending=False
    )[[
       'State/UnionTerritory',
       'Deaths','Confirmed',
       'FatalityRate'
       ]].style.background_gradient(
           cmap='Oranges'
       ).set_table_styles([
                           {'selector':'th',
                            'props':[('font-size','12px'),
                                     ('font-weight','bold')]},
                           {'selector':'td',
                            'props':[('font-size','11px'),
                                     ('font-weight','normal')]}
       ])

,State/UnionTerritory,Deaths,Confirmed,FatalityRate
2132,West Bengal,250,2961,0.084431
2121,Meghalaya,1,13,0.076923
2110,Gujarat,719,12140,0.059226
2124,Puducherry,1,18,0.055556
2118,Madhya Pradesh,258,5465,0.047210
2119,Maharashtra,1325,37136,0.035680
2112,Himachal Pradesh,3,92,0.032609
2115,Karnataka,40,1397,0.028633
2103,Assam,4,142,0.028169
2131,Uttar Pradesh,123,4926,0.024970


In [19]:
 crnt.sort_values(
     'RecoveryRate',ascending=False
     )[[
        'State/UnionTerritory',
        'Cured',
        'Confirmed',
        'RecoveryRate'
     ]].style.background_gradient(
         cmap='Greens'
     ).set_table_styles([
                         {'selector':'th',
                          'props':[
                                   ('font-size','12px'),
                                   ('font-weight','bold')
                          ]},
                         {'selector':'td',
                          'props':[
                                   ('font-size','11px'),
                                   ('font-weight','normal')
                          ]}
     ])

,State/UnionTerritory,Cured,Confirmed,RecoveryRate
2100,Andaman and Nicobar Islands,33,33,1.000000
2122,Mizoram,1,1,1.000000
2117,Ladakh,43,43,1.000000
2102,Arunachal Pradesh,1,1,1.000000
2121,Meghalaya,12,13,0.923077
2125,Punjab,1642,2002,0.820180
2116,Kerala,497,642,0.774143
2129,Tripura,116,173,0.670520
2111,Haryana,627,964,0.650415
2101,Andhra Pradesh,1621,2532,0.640205


In [25]:
mask = []

for state in cen['State / Union Territory'].unique():
  if state in cov['State/UnionTerritory'].unique():
    mask.append(False)
  else:
    mask.append(True)

cen[mask]['State / Union Territory']

27         Sikkim
35    Lakshadweep
Name: State / Union Territory, dtype: object

In [34]:
pd.merge(cov,cen,on='State/UnionTerritory',how='outer').head()

,Date,State/UnionTerritory,Cured,Deaths,Confirmed,Population,Area,Density,GenderRatio
0,30/01/20,Kerala,0.0,0.0,1.0,33406061.0,"38,863 km2 (15,005 sq mi)","859/km2 (2,220/sq mi)",1084.0
1,31/01/20,Kerala,0.0,0.0,1.0,33406061.0,"38,863 km2 (15,005 sq mi)","859/km2 (2,220/sq mi)",1084.0
2,01/02/20,Kerala,0.0,0.0,2.0,33406061.0,"38,863 km2 (15,005 sq mi)","859/km2 (2,220/sq mi)",1084.0
3,02/02/20,Kerala,0.0,0.0,3.0,33406061.0,"38,863 km2 (15,005 sq mi)","859/km2 (2,220/sq mi)",1084.0
4,03/02/20,Kerala,0.0,0.0,3.0,33406061.0,"38,863 km2 (15,005 sq mi)","859/km2 (2,220/sq mi)",1084.0
